In [32]:
import pandas as pd

In [33]:
file_path = '/content/sales_06_FY2020-21.csv'
data = pd.read_csv(file_path)

<ipython-input-33-488722885abf>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [34]:
data.head()

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
0,100354678,2020-10-01,received,574772.0,oasis_Oasis-064-36,21.0,89.9,1798.0,0.0,1798.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
1,100354678,2020-10-01,received,574774.0,Fantastic_FT-48,11.0,19.0,190.0,0.0,190.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
2,100354680,2020-10-01,complete,574777.0,mdeal_DMC-610-8,9.0,149.9,1199.2,0.0,1199.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
3,100354680,2020-10-01,complete,574779.0,oasis_Oasis-061-36,9.0,79.9,639.2,0.0,639.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
4,100367357,2020-11-13,received,595185.0,MEFNAR59C38B6CA08CD,2.0,99.9,99.9,0.0,99.9,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0


In [35]:
order_days_count = data.groupby('ref_num')['order_date'].nunique()

data['customer_return'] = data['ref_num'].apply(lambda x: 1 if order_days_count[x] >= 2 else 0)

data[['ref_num', 'order_date', 'customer_return']].head()


,ref_num,order_date,customer_return
0,987867,2020-10-01,1
1,987867,2020-10-01,1
2,987867,2020-10-01,1
3,987867,2020-10-01,1
4,987867,2020-11-13,1


In [36]:
data['customer_return'].value_counts()

,count
customer_return,
1,200935
0,85457


In [37]:
status_values = data['status'].unique()

print(status_values)

['received' 'complete' 'order_refunded' 'canceled' 'refund' 'cod' 'paid'
 'processing' 'closed' 'pending' 'pending_paypal' 'payment_review'
 'holded']


In [38]:
def classify_payment_status(status):
    if status in ['paid', 'complete', 'order_refunded', 'closed', 'received']:
        return 'paid'
    else:
        return 'unpaid'

data['payment_status'] = data['status'].apply(classify_payment_status)

payment_status_count = data['payment_status'].value_counts()

print(payment_status_count)


payment_status
paid      167388
unpaid    119004
Name: count, dtype: int64


In [39]:
data.head()

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,...,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent,customer_return,payment_status
0,100354678,2020-10-01,received,574772.0,oasis_Oasis-064-36,21.0,89.9,1798.0,0.0,1798.0,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,paid
1,100354678,2020-10-01,received,574774.0,Fantastic_FT-48,11.0,19.0,190.0,0.0,190.0,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,paid
2,100354680,2020-10-01,complete,574777.0,mdeal_DMC-610-8,9.0,149.9,1199.2,0.0,1199.2,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,paid
3,100354680,2020-10-01,complete,574779.0,oasis_Oasis-061-36,9.0,79.9,639.2,0.0,639.2,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,paid
4,100367357,2020-11-13,received,595185.0,MEFNAR59C38B6CA08CD,2.0,99.9,99.9,0.0,99.9,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,paid


In [40]:
file_path = '/content/sample_data/sales_06_FY2020-21_v2.csv'
data.to_csv(file_path, index=False)

# Phần dưới chưa làm không cần để ý

In [29]:
# Bước 1: Tiền xử lý dữ liệu
# Tính toán tần suất mua hàng
data['order_date'] = pd.to_datetime(data['order_date'])
frequency = data.groupby('ref_num')['order_date'].nunique().reset_index()
frequency.columns = ['ref_num', 'purchase_frequency']

# Tổng giá trị đơn hàng cho mỗi khách hàng
value_summary = data.groupby('ref_num')['value'].sum().reset_index()
value_summary.columns = ['ref_num', 'total_value']

# Kết hợp dữ liệu
customer_data = pd.merge(frequency, value_summary, on='ref_num')

# Bước 2: Chuẩn hóa dữ liệu
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(customer_data[['purchase_frequency', 'total_value']])

# Bước 3: Phân cụm với K-Means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)  # Chọn số cụm
customer_data['cluster'] = kmeans.fit_predict(scaled_data)

# Xem kết quả phân cụm
print(customer_data.head())


   ref_num  purchase_frequency  total_value  cluster
0   111127                   1      525.000        0
1   111143                   1      500.000        0
2   111197                  12     2620.275        0
3   111213                   2      872.000        0
4   111226                   1      114.800        0


In [30]:
# Đếm số lượng khách hàng trong từng cụm
cluster_counts = customer_data['cluster'].value_counts().reset_index()
cluster_counts.columns = ['cluster', 'customer_count']

# Tính giá trị trung bình của mỗi cụm
average_values = customer_data.groupby('cluster')['total_value'].mean().reset_index()
average_values.columns = ['cluster', 'average_value']

# Kết hợp thông tin
cluster_analysis = pd.merge(cluster_counts, average_values, on='cluster')

# Xem kết quả phân tích cụm
print(cluster_analysis)


   cluster  customer_count  average_value
0        0           61198   2.855454e+03
1        1             860   8.055691e+04
2        2               7   1.383127e+06


In [31]:
data

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,...,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent,customer_return,payment_status
0,100354678,2020-10-01,received,574772.0,oasis_Oasis-064-36,21.0,89.9,1798.0,0.0,1798.0,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,Đã trả tiền
1,100354678,2020-10-01,received,574774.0,Fantastic_FT-48,11.0,19.0,190.0,0.0,190.0,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,Đã trả tiền
2,100354680,2020-10-01,complete,574777.0,mdeal_DMC-610-8,9.0,149.9,1199.2,0.0,1199.2,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,Đã trả tiền
3,100354680,2020-10-01,complete,574779.0,oasis_Oasis-061-36,9.0,79.9,639.2,0.0,639.2,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,Đã trả tiền
4,100367357,2020-11-13,received,595185.0,MEFNAR59C38B6CA08CD,2.0,99.9,99.9,0.0,99.9,...,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0,1,Đã trả tiền
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286387,100562365,2021-09-30,paid,905179.0,APPCHA5AF14939B8F8A,2.0,4419.9,4419.9,0.0,4419.9,...,Rushville,Yates,Rushville,NY,14544,Northeast,bklatham,0.0,0,Đã trả tiền
286388,100562376,2021-09-30,cod,905191.0,MEFCOT5A8D1E973B886,2.0,39.9,39.9,0.0,39.9,...,Lawrenceville,Gwinnett,Lawrenceville,GA,30044,South,bmbrunetti,0.0,0,Chưa trả tiền
286389,100562383,2021-09-30,cod,905200.0,WOFVAL59D5EA84167F9-M,2.0,40.0,40.0,0.0,40.0,...,Durham,Durham,Durham,NC,27701,South,fngiusti,0.0,0,Chưa trả tiền
286390,100562384,2021-09-30,cod,905202.0,WOFNIG5B4D7EB0E9FDD-L,2.0,49.9,49.9,0.0,49.9,...,Durham,Durham,Durham,NC,27701,South,fngiusti,0.0,0,Chưa trả tiền
